In [18]:
from functions import get_points, write_to_laz, find_plane, find_clusters, kmeans_clusters, recursive_planes, filter_distance, hausdorff_distance, get_relevant_cids, find_distances_centroid

import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn

import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN

from scipy.spatial import cKDTree

from pyntcloud.ransac.fitters import single_fit
from pyntcloud.ransac.models import RansacPlane

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# test_area = [117813.6,487266.1,117942.1,487370.7]
# xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
# wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
# wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # stkuje brug


xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc18 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 2018')

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc19 = get_points(path_a10_2019, bounds, wkt)

print(f'loaded pc 2019')

loaded pc 2018
loaded pc 2019


In [12]:
# from functions import filter_distance 

filtered_2019 = filter_distance(pc19, pc18)
print('filtered')
points_2019 = pd.DataFrame(
    np.array(filtered_2019[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

# from functions import filter_distance 

filtered_2018 = filter_distance(pc18, pc19)
print('filtered')
points_2018 = pd.DataFrame(
    np.array(filtered_2018[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

filtered
dataframed
filtered
dataframed


In [13]:
try:
    del points_with_planes
except:
    pass
planes18, models18 = recursive_planes(points_2018.copy(), 5, 200)
print('2018 done')
planes19, models19 = recursive_planes(points_2019.copy(), 5, 200)
print('2019 done')

clustering cluster 1
clustering cluster 2
clustering cluster 3
clustering cluster 4
clustering cluster 5
2018 done
clustering cluster 1
clustering cluster 2
clustering cluster 3
clustering cluster 4
clustering cluster 5
2019 done


In [15]:
rel_cids18 = get_relevant_cids(planes18)
rel_cids19 = get_relevant_cids(planes19)

In [16]:
for i in range(2):
    print('2018:', rel_cids18[i], models18[rel_cids18[i]].point, models18[rel_cids18[i]].normal)
    print('2019:', rel_cids19[i], models19[rel_cids19[i]].point, models19[rel_cids19[i]].normal)
    
    

2018: 1 [ 1.17893747e+05  4.87315534e+05 -1.47632787e-01] [ 0.03238913 -0.01177219  0.999406  ]
2019: 1 [ 1.17892617e+05  4.87318483e+05 -1.02484127e-01] [ 0.02760994 -0.01271527  0.9995379 ]
2018: 3 [1.17891750e+05 4.87317474e+05 4.76712930e+00] [-0.00343322 -0.0153762   0.99987588]
2019: 2 [1.17891838e+05 4.87318254e+05 4.76943928e+00] [-0.00711603 -0.01502197  0.99986184]


In [ ]:
ransac = rfn.append_fields(filtered_2019[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              planes19['cid'].astype(np.uint16),
                              usemask=False
                             )

write_to_laz(ransac, '/var/data/rws/data/output/tst_ransac/portal_planes.laz')
print('wrote')

In [28]:
z_2018 = find_distances_centroid(models18[rel_cids18[0]], models18[rel_cids18[1]], shapely.wkt.loads(wkt))
z_2019 = find_distances_centroid(models19[rel_cids19[0]], models19[rel_cids19[1]], shapely.wkt.loads(wkt))

In [29]:
print(z_2018, '\t' , z_2019, z_2018 - z_2019)

4.78606885341069 	 4.809754589391283 -0.023685735980593314
